# Covid Tweet Legitmacy Classifier

In [1]:
# the model we gonna train, Covid-Twitter-Bert V2
# check text classification models here: https://huggingface.co/models?filter=text-classification
# https://huggingface.co/digitalepidemiologylab/covid-twitter-bert-v2
model_name = "digitalepidemiologylab/covid-twitter-bert-v2"
tokenizer_name = "digitalepidemiologylab/covid-twitter-bert-v2"
max_length = 96

### Imports

In [2]:
import numpy as np
import pandas as pd
import regex as re
import string

import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AdamW,
    Trainer,
    TrainingArguments
)

import random
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

### Set seed for consistency

In [3]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(5)     # For 'first-augmented-miscov19-covid-twitter-bert-v2' model
# set_seed(314)   # For 'second-augmented-miscov19-covid-twitter-bert-v2' model
# set_seed(2718)  # For 'third-augmented-miscov19-covid-twitter-bert-v2' model
# set_seed(4669)  # For 'fourth-augmented-miscov19-covid-twitter-bert-v2' model

### Load data and create the datasets

In [4]:
raw_data = pd.read_csv('miscov19_augmented_final_hydrated.csv')
df = raw_data[['text','label']]
df.dropna()
df['text'] = df['text'].astype(str)
df.tail()

/tmp/ipykernel_12006/3992126773.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].astype(str)


,text,label
6572,nan,1
6573,Why is everyone pretending like theyre immune ...,0
6574,CDC said if you snort enough cocaine it would ...,2
6575,A message hailing the powers of 'boiled garlic...,0
6576,5G Networks? Bleach? Ultraviolet light? The co...,0


In [5]:
target_names = ['legitimate','misinformation','irrelevant']

In [6]:
stop = stopwords.words('english')

In [7]:
def clean_text(row):
    # Lower case
    row = row.lower()
    
    # Remove URLs
    row = re.sub('http\S+|www.\S+', '', row)
    
    # Remove @mentions
    row = re.sub('@[A-Za-z0-9]+', '', row)
    
    # Remove non-standard characters
    row = row.encode("ascii", "ignore").decode()
    
    # Remove punctuation
    row = row.translate(str.maketrans('', '', string.punctuation))
    
    # Remove stop words
    pat = r'\b(?:{})\b'.format('|'.join(stop))
    row = row.replace(pat, '')
    row = row.replace(r'\s+', ' ')
    
    # Remove extraneous whitespace
    row = row.strip()
    
    # Lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    w_tokenization = nltk.word_tokenize(row)
    final = ""
    for w in w_tokenization:
        final = final + " " + wordnet_lemmatizer.lemmatize(w)
    
    return final

df['text'] = df['text'].apply(clean_text)

/tmp/ipykernel_12006/3244438452.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_text)


In [8]:
documents = df['text'].tolist()
labels = df['label'].tolist()

In [9]:
test_size = 0.2 # Percentage of dataset used for validation

In [10]:
train_texts, valid_texts, train_labels, valid_labels = train_test_split(documents, labels, test_size=test_size)

In [11]:
# Use only if training notebook from scratch
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

In [12]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

In [13]:
class MiscovDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
# convert our tokenized data into a torch Dataset
train_dataset = MiscovDataset(train_encodings, train_labels)
valid_dataset = MiscovDataset(valid_encodings, valid_labels)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")

Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert-v2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClass

In [16]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

In [17]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate = 1e-5,            # learning rate
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    metric_for_best_model='accuracy',
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [18]:
trainer.train()

***** Running training *****
  Num examples = 5261
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3948


Step,Training Loss,Validation Loss,Accuracy
400,1.065000,0.968088,0.531915
800,0.842900,0.701968,0.718845
1200,0.693700,0.620640,0.771277
1600,0.621300,0.669740,0.775076
2000,0.576000,0.781552,0.778116
2400,0.661700,0.674528,0.784195
2800,0.503100,0.731292,0.781155
3200,0.382100,0.845506,0.787234
3600,0.498600,0.844073,0.775836


***** Running Evaluation *****
  Num examples = 1316
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1316
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-800
Configuration saved in ./results/checkpoint-800/config.json
Model weights saved in ./results/checkpoint-800/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1316
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1200
Configuration saved in ./results/checkpoint-1200/config.json
Model weights saved in ./results/checkpoint-1200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1316
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1600
Configuration saved in ./results/checkpoint-1600/config.json
Model weights saved in ./results/checkpoint-1600/pytorch_model.bi

TrainOutput(global_step=3948, training_loss=0.6327137739341914, metrics={'train_runtime': 1577.7288, 'train_samples_per_second': 10.004, 'train_steps_per_second': 2.502, 'total_flos': 2269509902321058.0, 'train_loss': 0.6327137739341914, 'epoch': 3.0})

In [19]:
# evaluate the current model after training
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1316
  Batch size = 20


{'eval_loss': 0.8455063104629517,
 'eval_accuracy': 0.7872340425531915,
 'eval_runtime': 21.7769,
 'eval_samples_per_second': 60.431,
 'eval_steps_per_second': 3.031,
 'epoch': 3.0}

In [20]:
# saving the fine tuned model & tokenizer
model_path = "first-augmented-miscov19-covid-twitter-bert-v2"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in sixth-augmented-miscov19-covid-twitter-bert-v2/config.json
Model weights saved in sixth-augmented-miscov19-covid-twitter-bert-v2/pytorch_model.bin
tokenizer config file saved in sixth-augmented-miscov19-covid-twitter-bert-v2/tokenizer_config.json
Special tokens file saved in sixth-augmented-miscov19-covid-twitter-bert-v2/special_tokens_map.json


('sixth-augmented-miscov19-covid-twitter-bert-v2/tokenizer_config.json',
 'sixth-augmented-miscov19-covid-twitter-bert-v2/special_tokens_map.json',
 'sixth-augmented-miscov19-covid-twitter-bert-v2/vocab.txt',
 'sixth-augmented-miscov19-covid-twitter-bert-v2/added_tokens.json',
 'sixth-augmented-miscov19-covid-twitter-bert-v2/tokenizer.json')

In [21]:
def get_prediction(text):
    # apply preprocessing to text
    inputs = text # clean_text(text)
    # prepare our text into tokenized sequence
    inputs = tokenizer(inputs, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]

In [22]:
# Example #1
text = "DP Dough is the best restaurant in New York"
print(get_prediction(text))

irrelevant


In [23]:
# Example #2
text2 = "Vaccines cause autism"
print(get_prediction(text2))

misinformation


In [24]:
# Example #3
text3 = "Vaccinations prevent over 90% of Covid infections! #Science"
print(get_prediction(text3))

legitimate


In [25]:
# Example #4
text4 = "Vaccines will end the pandemic"
print(get_prediction(text4))

legitimate


In [26]:
# Example #5
text5 = "scientists say bleach will prevent covid"
print(get_prediction(text5))

misinformation


In [27]:
# Example #6
text6 = "Biden says vaccines prevent over 90% of Covid infections!"
print(get_prediction(text6))

misinformation


In [28]:
# Example #7
text7 = "Biden says vaccines cause autism!"
print(get_prediction(text7))

misinformation


In [29]:
# Example #8
text8 = "In Portugal, with 89% of the total population fully vaccinated, almost 90% of UCI Covid patients are unvaccinated"
print(get_prediction(text8))

legitimate


In [30]:
# Example #9
text9 = "President Trump has covid"
print(get_prediction(text9))

irrelevant


In [31]:
# Example #10
text10 = "Vaccines don't stop you from getting covid."
print(get_prediction(text10))

legitimate


In [32]:
# Example #11
text11 = "Vaccinations stop you from getting covid."
print(get_prediction(text11))

misinformation
